In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.errors import AnalysisException

from dotenv import load_dotenv
load_dotenv("../.env-deploy", override=True)

True

In [2]:
data_home = "/Users/kwesi/Desktop/ai/gpts/mlsgpt/data"
jar_files = ["postgresql-42.7.3.jar", "mysql-connector-j-8.0.33.jar"]
jar_opts = ",".join([f"{data_home}/jars/{jar}" for jar in jar_files])
warehouse = f"{data_home}/warehouse"

spark: SparkSession = (
    SparkSession.builder\
    .appName("MLSGPT")
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.shuffle.service.enabled", "true")
    .config("spark.sql.warehouse.dir", f"{warehouse}")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.jars", f"{jar_opts}") 
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

24/05/17 01:12:05 WARN Utils: Your hostname, marley.local resolves to a loopback address: 127.0.0.1; using 10.0.0.135 instead (on interface en0)
24/05/17 01:12:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/17 01:12:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
my_url = "jdbc:mysql://{}:{}/{}".format(os.getenv("MYSQL_HOST"), os.getenv("MYSQL_PORT"),os.getenv("MYSQL_DB"))
my_props = {
    "user": os.getenv("MYSQL_USER"),
    "password": os.getenv("MYSQL_PASSWORD"),
    "driver": "com.mysql.cj.jdbc.Driver"
}

pg_url = "jdbc:postgresql://{}:{}/{}".format(os.getenv("POSTGRES_HOST"), os.getenv("POSTGRES_PORT"),os.getenv("POSTGRES_DB"))
pg_props = {
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
    "driver": "org.postgresql.Driver"
}

In [4]:
boards_df = spark.read.jdbc(url=my_url, table="rsbr0_rps_boards", properties=my_props)
office_df = spark.read.jdbc(url=my_url, table="rsbr0_rps_office", properties=my_props)
agent_df = spark.read.jdbc(url=my_url, table="rsbr0_rps_agent", properties=my_props)
property_df = spark.read.jdbc(url=my_url, table="rsbr0_rps_property", properties=my_props)
property_rooms_df = spark.read.jdbc(url=my_url, table="rsbr0_rps_property_rooms", properties=my_props)

In [5]:
for df in [boards_df, office_df, agent_df, property_df, property_rooms_df]:
    df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- OrganizationID: integer (nullable = true)
 |-- ShortName: string (nullable = true)
 |-- LongName: string (nullable = true)

root
 |-- office_id: long (nullable = true)
 |-- OfficeID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- ID: long (nullable = true)
 |-- LastUpdated: string (nullable = true)
 |-- LogoLastUpdated: string (nullable = true)
 |-- Logos: string (nullable = true)
 |-- OrganizationType: string (nullable = true)
 |-- Designation: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Franchisor: string (nullable = true)
 |-- StreetAddress: string (nullable = true)
 |-- AddressLine1: string (nullable = true)
 |-- AddressLine2: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Province: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- AdditionalStreetInfo: string (nullable = true)
 |-- CommunityName: string (nullable = t

24/05/17 04:42:17 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [ ]:
# dfs = {
#     "boards": boards_df,
#     "office": office_df,
#     "agent": agent_df,
#     "property": property_df,
#     "property_rooms": property_rooms_df

# }
# for k, v in dfs.items():
#     print(f"Schema for {k}")
#     print(20*"=")
#     for schema in v.schema:
#         if schema.dataType == T.BinaryType():
#             print(f"Decoding {schema.name} in {k}")
#             dfs[k] = v.withColumn(schema.name, F.cast(T.StringType, F.decode(schema.name, "utf-8")).alias(schema.name))
#     print("\n")

In [ ]:
# schema = "rsbr_decoded"
# for k, v in dfs.items():
#     v.write.jdbc(url=pg_url, table=f"{schema}.{k}", mode="overwrite", properties=pg_props)